In [1]:
import os
import sys
import glob
from os.path import abspath
os.environ['SPARK_HOME'] = 'C:\spark-3.1.2-bin-hadoop3.2'
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk1.8.0_201'
os.environ['HADOOP_HOME'] = 'C:\spark-3.1.2-bin-hadoop3.2'
spark_python = os.path.join(os.environ.get('SPARK_HOME',None),'python')
py4j = glob.glob(os.path.join(spark_python,'lib','py4j-*.zip'))[0]
sys.path[:0]=[spark_python,py4j]
os.environ['PYTHONPATH']=py4j                          
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark Examples").enableHiveSupport().getOrCreate()

In [2]:
from pyspark.sql.types import StructType
temperatureSchema = StructType().add("day", "string").add("tempInCelcius", "string")

In [3]:
temperature_streaming_df = spark.readStream.option("sep", ",") \
                            .schema(temperatureSchema).csv("C:\\Users\\ADMIN\\Documents\\temperature-data")

In [4]:
temperature_streaming_df.isStreaming

True

In [5]:
temperature_streaming_df.printSchema()

root
 |-- day: string (nullable = true)
 |-- tempInCelcius: string (nullable = true)



In [6]:
query = temperature_streaming_df.writeStream.format("console").outputMode("append").start()

In [7]:
query = temperature_streaming_df.writeStream.format("memory").queryName("TemperatureData").outputMode("append").start()

In [8]:
spark.sql("select * from TemperatureData").show()

+-----+-------------+
|  day|tempInCelcius|
+-----+-------------+
|  day|tempInCelsius|
| day8|         12.3|
| day9|         13.1|
|day10|         12.9|
|day11|         11.9|
|day12|           14|
|day13|         13.9|
|day14|         12.7|
|  day|tempInCelsius|
| day8|         12.3|
| day9|         13.1|
|day10|         12.9|
|day11|         11.9|
|day12|           14|
|day13|         13.9|
|day14|         12.8|
|  day|tempInCelsius|
| day1|         12.3|
| day2|         13.1|
| day3|         12.9|
+-----+-------------+
only showing top 20 rows



In [12]:
spark.sql("select * from TemperatureData where day != 'day' and cast(tempInCelcius as double) >13").show()

+-----+-------------+
|  day|tempInCelcius|
+-----+-------------+
| day9|         13.1|
|day12|           14|
|day13|         13.9|
| day9|         13.1|
|day12|           14|
|day13|         13.9|
| day2|         13.1|
| day5|           14|
| day6|         13.9|
+-----+-------------+

